This notebook will first explain how does `dataprep.connector` get the data using configs.

After that, we will demonstrate how to create a new config to get data from the `earnings_calendar` API endpoint using `dataprep.connector` and do visualization on the data using `dataprep.eda`.

# Config

`dataprep.connector` is a component that simplifies the API call. It first reads a config, which describes how to call a specific API, and then lets you make API calls using a simple function.

An example config for downloading the [earnings_calendar](https://finnhub.io/docs/api/earnings-calendar) information from the [finnhub](https://finnhub.io/) website is already provided in the `finnhub` folder.

A config is a folder whose name is usually the website name. In our case, it's `finnhub`.

The folder consists of at least two files, a `_meta.json` which describes the names of the endpoints under this website,
and an API description file to describe **one** API endpoint information (if you want to describe different API endpoints under the same website, that will be multiple API description files).

Currently, the API description file for the `countries` API endpoint is provided in the `finnhub/earnings_calendar.json` file.

### _meta.json

In [ ]:
# The following code outputs the content of the _meta.json
!cat finnhub/_meta.json

First, take a look at the content of "_meta.json", it is a JSON object, which has only one field called "tables". Under "tables" is an array of strings, where each of them is the name of an API endpoint. Later in the hackathon, if you'd like to create a config for a website, don't forget to create the `_meta.json` file and add the endpoint names into the file.

`dataprep.connector` will first read this file when you call `connect("finnhub")` to know what are the names of all the API endpoints under this website, as well as reading all the API descriptions for each `name` from the `{name}.json` file. For example, here under the `tables` field, there's one element called `earnings_calendar`. As a result, `connect` will read a file called `earnings_calendar.json` in the same folder as `_meta.json`.

You may notice here we name the the API endpoints as tables. This is because, for these endpoints, they return a bunch of records that resembles a database table.

### API Description

In [ ]:
# The following code outputs the content of the countries.json
!cat finnhub/earnings_calendar.json

As mentioned in the video, the API description file consists of three main fields:
```json
{
    "version": 1,
    "request": {},
    "response": {}
}

```

The `version` field indicates the version of this config file format, for now, it will always be 1.

The `request` field is an object, which describes how to make a request to an endpoint. Specifically, it describes the endpoint URL, the HTTP method, and the parameters required.

The `response` field is also an object, which describes how will connector parse the response from the API endpoint. It describes the fields the response contains, as well as the format of the response.

### API Description Explained

There are two fields in the API description requires some elaboration, the `request` and `response`.

#### Request

```json
"request": {
    "url": "https://finnhub.io/api/v1/calendar/earnings",
    "method": "GET",
    "authorization": {
        "type": "QueryParam",
        "keyParam": "token"
    },
    "params": {
        "symbol": false,
        "from_": false,
        "from": {
            "required": false,
            "removeIfEmpty": true,
            "template": "{{from_}}",
            "fromKey": [
                "from_"
            ]
        },
        "to": false,
        "international": false
    }
}
```

The request field describes how to issue a request to the API endpoint. 

Usually you can get these information from the API documentation. For example, you can get the information for `earnings_calendar` [here](https://finnhub.io/docs/api/earnings-calendar).

The request field at least should contain the following fields:
    
1. `url` field, which indicates the location of the endpoint.
2. `method` field, which indicates the HTTP method to issue the request. Usually, this will be `GET`.
3. `authorization` field. This field is optional and is only required if the website needs some form of authorization. For example, 
   finnhub requires the `QueryParam` authorization type. Essentially this means the API key should be put in the URL as a query parameter. The name of the parameter is
   indicated using `keyParam` field. In our case, it is called `token.
4. `params` defines all the parameters the API request required. They can be mandatory or optional. The simplest form of `params` is `"parameter name": required or not`. 
   For example, `"symbol": false` means the endpoint takes a parameter called `symbol`, and it is optional.
   A more complex form is like this
   ```json
    "from": {
        "required": false,
        "removeIfEmpty": true,
        "template": "{{from_}}",
        "fromKey": [
            "from_"
        ]
    },
    ```
    which basically says that there's a parameter called from, it is not required, and it should be removed if the value of it is empty. It is a template parameter, where it takes the value of another parameter called `from_` (remind that `from` is the Python keyword so we cannot use it as the variable name. That's why we define the query parameter `from` gets the value from `from_`). `fromKey` indicates `from` is dependent from another parameter called `from_`.
  


#### Response

The following is the `response` field (some fields are elided for easy reading).

```json
"response": {
    "ctype": "application/json",
    "tablePath": "$.earningsCalendar[*]",
    "schema": {
        "date": {
            "target": "$.date",
            "type": "string"
        },
        "epsActual": {
            "target": "$.epsActual",
            "type": "float"
        },
        ...
    },
    "orient": "records"
}
```

Response field describes how to parse data from the response of the endpoint.

There are four subfields required:

1. `ctype` indicates the API response format. Keep this as `application/json` since usually, a website returns JSON as the response.
2. `orient`, `tablePath` and `schema` require some understanding of the response. For example, from the finnhub [documentation](https://finnhub.io/docs/api/earnings-calendar), the response of 
   earnings calendar looks like this:
   ```json
   {
      "earningsCalendar": [
        {
          "date": "2020-01-28",
          "epsActual": 4.99,
          "epsEstimate": 4.5474,
          "hour": "amc",
          "quarter": 1,
          "revenueActual": 91819000000,
          "revenueEstimate": 88496400810,
          "symbol": "AAPL",
          "year": 2020
        },
        {
          "date": "2019-10-30",
          "epsActual": 3.03,
          "epsEstimate": 2.8393,
          "hour": "amc",
          "quarter": 4,
          "revenueActual": 64040000000,
          "revenueEstimate": 62985161760,
          "symbol": "AAPL",
          "year": 2019
        }
       ]
    }
   ```
   We see that the response is a JSON object, which contains a single field `earningsCalendar`. Under this field, it is an array of objects, and each object is a record.
   The way that uses an array of objects to represent the content is called `records`. As a result, the `orient` field should have the value `records`.
   
3. Next, since the array of objects is under the key `earningsCalendar`, we use `"tablePath": "$.earningsCalendar[*]"` to indicate the location of the data records. 
   The syntax of `$.earningsCalendar[*]` is called JSONPath. You can find the detail at https://github.com/h2non/jsonpath-ng.
 
4. Finally, we write down the description of each field under the `schema` field, with their name and the JSONPath with respect to the `tablePath`. 
   For example, for the `date` field in the response we create an object under the `schema` field in the endpoint description as
   ```json
   "date": {
        "target": "$.date",
        "type": "string"
    },
   ```

Now you must have a good understanding on the config file format. 

Next we are going to show an example workflow for EZHacks based on the `earnings_calendar` API.

Specifically, we will do a step by step demostration on how to create a config and get data from the `earnings_calendar` API, and then do a visualization.

# Creating a new config file

Suppose during the hackathon, you want to do some investigation on investing US stock market.
As the first step, you'd like to see the revenue trend of some tech companies, for example, TESLA.

# Plan

Following is the plan to finish the task:

1. Find a website that provides financial data API.
2. Use `dataprep.connector` to download the data.
3. Use `dataprep.eda` to visualize the data.

## Finnhub

After some investigation, you decide to use the [Finnhub](https://finnhub.io/) website, where they provide
API access to fetch the financial related data.

The most related one is the [earnings calendar](https://finnhub.io/docs/api/earnings-calendar) API, where it provides the quarterly revenue data of a company.

Reading through their [documentation](https://finnhub.io/docs/api/authentication), you notice the first thing you need to do is creating an account
because all the API calls to finnhub require an API key.

Once registered, go to the [dashboard](https://finnhub.io/dashboard) and write down the API key.

### Create a config for the earnings calendar endpoint

The first thing to do is adding the endpoint name "earnings_calendar" to the `_meta.json`.

The content of `_meta.json` will become this:
```json
{
    "tables": [
        "earnings_calendar"
    ]
}
```

In [ ]:
%%bash

# The following code adds the content above to finnhub/_meta.json

cat > finnhub/_meta.json << EOF
{
    "tables": [
        "earnings_calendar"
    ]
}
EOF

Next, create a new file called `earnings_calendar.json`.

Put the following content into `earnings_calendar.json`.

```json
{
    "version": 1,
    "request": {
        "url": "https://finnhub.io/api/v1/calendar/earnings",
        "method": "GET",
        "authorization": {
            "type": "QueryParam",
            "keyParam": "token"
        },
        "params": {
            "symbol": false,
            "from_": false,
            "from": {
                "required": false,
                "removeIfEmpty": true,
                "template": "{{from_}}",
                "fromKey": [
                    "from_"
                ]
            },
            "to": false,
            "international": false
        }
    },
    "response": {
        "ctype": "application/json",
        "tablePath": "$.earningsCalendar[*]",
        "schema": {
            "date": {
                "target": "$.date",
                "type": "string"
            },
            "epsActual": {
                "target": "$.epsActual",
                "type": "float"
            },
            "epsEstimate": {
                "target": "$.epsEstimate",
                "type": "float"
            },
            "hour": {
                "target": "$.hour",
                "type": "string"
            },
            "quarter": {
                "target": "$.quarter",
                "type": "int"
            },
            "revenueActual": {
                "target": "$.revenueActual",
                "type": "float"
            },
            "revenueEstimate": {
                "target": "$.revenueEstimate",
                "type": "float"
            },
            "symbol": {
                "target": "$.symbol",
                "type": "string"
            },
            "year": {
                "target": "$.year",
                "type": "int"
            }
        },
        "orient": "records"
    }
}
```

In [ ]:
%%bash

# The following code adds the content above to finnhub/earnings_calendar.json

cat > finnhub/earnings_calendar.json << EOF
{
    "version": 1,
    "request": {
        "url": "https://finnhub.io/api/v1/calendar/earnings",
        "method": "GET",
        "authorization": {
            "type": "QueryParam",
            "keyParam": "token"
        },
        "params": {
            "symbol": false,
            "from_": false,
            "from": {
                "required": false,
                "removeIfEmpty": true,
                "template": "{{from_}}",
                "fromKey": [
                    "from_"
                ]
            },
            "to": false,
            "international": false
        }
    },
    "response": {
        "ctype": "application/json",
        "tablePath": "$.earningsCalendar[*]",
        "schema": {
            "date": {
                "target": "$.date",
                "type": "string"
            },
            "epsActual": {
                "target": "$.epsActual",
                "type": "float"
            },
            "epsEstimate": {
                "target": "$.epsEstimate",
                "type": "float"
            },
            "hour": {
                "target": "$.hour",
                "type": "string"
            },
            "quarter": {
                "target": "$.quarter",
                "type": "int"
            },
            "revenueActual": {
                "target": "$.revenueActual",
                "type": "float"
            },
            "revenueEstimate": {
                "target": "$.revenueEstimate",
                "type": "float"
            },
            "symbol": {
                "target": "$.symbol",
                "type": "string"
            },
            "year": {
                "target": "$.year",
                "type": "int"
            }
        },
        "orient": "records"
    }
}
EOF

Now run the following code

In [ ]:
# Import related functions
from dataprep.connector import connect
from datetime import datetime, timedelta, timezone

# connect to finnhub, the connector will automatically read the config from the finnhub folder. The access token is the API key you just acquired from the finnhub dashboard.
conn_finnhub = connect("./finnhub", _auth={"access_token":"put your finnhub API token here"})

# issue a query. The query consists of two things, the API endpoint name (earnings_calendar) and the parameters. From the finnhub documentation https://finnhub.io/docs/api/earnings-calendar
# we know that earnings_calendar takes four parameters, but now we only care about three of them (symbol, from, to). Since from is a python keyword, we rename it to from_.
# the await keyword is required because we are using the python async/await feature to issue HTTP requests.
tesla = await conn_finnhub.query('earnings_calendar', symbol='TSLA', from_=datetime(2011,1,1), to=datetime.now())

# Voila! the API response is parsed into a dataframe
tesla

## Plot and Inspect the Result

Since we are interested in the trend of the revenue of Tesla, we can use the `dataprep.eda` module to visualize the `year` and the `revenueActual` column. (Want to know where do these two column names come from? Check the DataFrame header of the previous cell!)

In [ ]:
# this will import the plot function from the eda module.
from dataprep.eda import plot

# The plot function takes at least one argument, which is a DataFrme, i.e. `plot(tesla)`. This will give you an overview of each column of the DataFrame.
# It can also take two or three arguments. For the following three-argument call, `plot` will give you a plot showing the relationship of two columns, which are passed as
# the second and the third parameter.

plot(tesla, "year", "revenueActual")

## Conclusion from the plot

By inspecting the plot, you found that the revenue of Tesla is increasing year by year, which indicates a potentially good investment target. 
However, if you take a look at the box plot, the variance of the recent year becomes bigger and bigger. 
You may think the variance of the company is too much, so you decide to do other investigations before making a final decision.